In [1]:
from scrape_stats_function import scrape_player_stats
import time
import pandas as pd
pd.set_option('display.max_columns', None)
'''
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        if 'stats_standard' in comment:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find('table', id='stats_standard')
'''

"\n    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):\n        if 'stats_standard' in comment:\n            comment_soup = BeautifulSoup(comment, 'html.parser')\n            table = comment_soup.find('table', id='stats_standard')\n"

In [2]:
years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016","2014-2015","2013-2014","2012-2013","2011-2012","2010-2011"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/stats/{year}-Bundesliga-Stats"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/stats/{year}-Premier-League-Stats"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/stats/{year}-La-Liga-Stats"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/stats/{year}-Serie-A-Stats"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/stats/{year}-Ligue-1-Stats"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/stats/{year}-Eredivisie-Stats"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/stats/{year}-Primeira-Liga-Stats"
    },
    {
        "name": "Championship",
        "url": "https://fbref.com/en/comps/10/{year}/stats/{year}-EFL-Championship-Stats"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/stats/{year}-Scottish-Premiership-Stats"
    }
 
]

leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
        })

league_dfs = {}
combined_list = []
for league in leagues:
    df = scrape_player_stats(league["name"], league["url"], table_id='stats_standard')
    if df is not None:
        league_dfs[league["name"]] = df
        combined_list.append(df)
    else:
        print(f"Failed for {league['name']} at {league['url']}")
    time.sleep(5)  

200 https://fbref.com/en/comps/20/2024-2025/stats/2024-2025-Bundesliga-Stats
Found stats_standard in comment for Bundesliga 2024-2025
200 https://fbref.com/en/comps/9/2024-2025/stats/2024-2025-Premier-League-Stats
Found stats_standard in comment for Premier League 2024-2025
200 https://fbref.com/en/comps/12/2024-2025/stats/2024-2025-La-Liga-Stats
Found stats_standard in comment for La Liga 2024-2025


KeyboardInterrupt: 

In [3]:
df = pd.concat(combined_list,ignore_index=True)


df2 = df[~df['Player'].str.contains("Player", case=False, na=False)]

#df_dropped_players.loc[df_dropped_players['90s'].idxmax()]


In [5]:
pd.set_option('display.max_columns', None)

df2 = df2.drop(['Matches','Rk'], axis=1)


In [6]:
int64_col = ['Born','Age','MP','Starts','Gls','Ast','G+A','G-PK','PK','PKatt','CrdY','CrdR','PrgC','PrgP','PrgR','Min']

float64_col = ['90s','xG','xAG','npxG','npxG+xAG','Gls.1','Ast.1','G-PK.1','G+A-PK','xG.1','xAG.1','xG+xAG','npxG.1','npxG+xAG.1','G+A.1']

for col in float64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce')

for col in int64_col:
    df2[col] = pd.to_numeric(df2[col], errors='coerce').astype('Int64')

df2 = df2.astype({
    'Player': 'string',
    'Nation': 'string',
    'Pos': 'string',
    'Squad': 'string',
    'League': 'string',
})


df2.isna().sum()

Player        0
Nation        6
Pos           0
Squad         0
Age           7
Born          7
MP            0
Starts        0
Min           0
90s           0
Gls           0
Ast           0
G+A           0
G-PK          0
PK            0
PKatt         0
CrdY          0
CrdR          0
xG            0
npxG          0
xAG           0
npxG+xAG      0
PrgC          0
PrgP          0
PrgR          0
Gls.1         0
Ast.1         0
G+A.1         0
G-PK.1        0
G+A-PK        0
xG.1          0
xAG.1         0
xG+xAG        0
npxG.1        0
npxG+xAG.1    0
League        0
dtype: int64

In [10]:
df2[df2['Age'].isna()]

new_columns = {
    'MP': 'Matches Played',
    'G-PK':'Non-Pen Gls',
    'PK': 'Pens Made',
    'PKatt': 'Pen Kicks Att',
    'CrdY': 'Yellow Cards',
    'CrdR': 'Red Cards',
    'PrgC': 'Prog Carries',
    'PrgP': 'Prog Passes',
    'PrgR': 'Prog Receives',
    'Gls.1': 'Gls/90',
    'Ast.1': 'Ast/90',
    'G+A.1': 'G+A/90',
    'G-PK.1': 'Non-Pen Gls/90',
    'G+A-PK': 'Non-Pen Gls+Ast-PK/90',
    'xG.1': 'xG/90',
    'xAG.1': 'xAG/90',
    'xG+xAG': 'xG+xAG/90',
    'npxG.1': 'npxG/90',
    'npxG+xAG.1': 'npxG+xAG/90',
}

df2.rename(columns=new_columns, inplace=True)

In [11]:
df2[df2['Nation'].isna()]

,Player,Nation,Pos,Squad,Age,Born,Matches Played,Starts,Min,90s,Gls,Ast,G+A,Non-Pen Gls,Pens Made,Pen Kicks Att,Yellow Cards,Red Cards,xG,npxG,xAG,npxG+xAG,Prog Carries,Prog Passes,Prog Receives,Gls/90,Ast/90,G+A/90,Non-Pen Gls/90,Non-Pen Gls+Ast-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90,League
215,Atakan Karazor,<NA>,MF,Stuttgart,27,1996,32,31,2618,29.1,2,4,6,2,0,0,10,1,2.3,2.0,2.6,4.6,19,140,33,0.07,0.14,0.21,0.07,0.21,0.08,0.09,0.17,0.07,0.16,Bundesliga 2024-2025
528,Olabade Aluko,<NA>,DF,Leicester City,<NA>,<NA>,1,0,2,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Premier League 2024-2025
688,Jake Evans,<NA>,FW,Leicester City,<NA>,<NA>,4,0,24,0.3,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Premier League 2024-2025
851,Mateus Mane,<NA>,MF,Wolves,<NA>,<NA>,1,0,2,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Premier League 2024-2025
884,Jeremy Monga,<NA>,"FW,MF",Leicester City,<NA>,<NA>,7,0,112,1.2,0,0,0,0,0,0,0,0,0.2,0.2,0.0,0.2,6,3,10,0.00,0.00,0.00,0.00,0.00,0.13,0.01,0.14,0.13,0.14,Premier League 2024-2025
1434,Fer López,<NA>,"FW,MF",Celta Vigo,<NA>,<NA>,17,7,671,7.5,2,0,2,2,0,0,2,0,2.7,2.7,1.9,4.6,27,43,58,0.27,0.00,0.27,0.27,0.27,0.37,0.25,0.62,0.37,0.62,La Liga 2024-2025


In [12]:
df2.loc[df2['Gls'].idxmin()]

Player                         Erik Ahlstrand
Nation                                 se SWE
Pos                                     FW,MF
Squad                               St. Pauli
Age                                        22
Born                                     2001
Matches Played                              5
Starts                                      0
Min                                        43
90s                                       0.5
Gls                                         0
Ast                                         0
G+A                                         0
Non-Pen Gls                                 0
Pens Made                                   0
Pen Kicks Att                               0
Yellow Cards                                0
Red Cards                                   0
xG                                        0.0
npxG                                      0.0
xAG                                       0.0
npxG+xAG                          